In [8]:
import os
import time
import pandas as pd
import pymysql
import numpy as np
from haversine import haversine
from decimal import Decimal
from tqdm.auto import tqdm

def live_db_conn():
    """Live DB 접속 함수"""
    conn = pymysql.connect(
        host = "db-6j3k3.pub-cdb.ntruss.com", 
        user = "redtable",
        password = "fpemxpdlqmf5491!@#",
        autocommit = True, 
        cursorclass = pymysql.cursors.DictCursor, 
        db = "redtable2021")
    return conn


def dev_db_conn():
    """개발 DB 접속 함수"""
    conn = pymysql.connect(
        host = 'db-7ma06.pub-cdb.ntruss.com',
        user = 'redtable',
        password = 'fpemxpdlqmf5491!@#',
        autocommit = True,
        cursorclass = pymysql.cursors.DictCursor)
    return conn


def rt_db_conn():
    """빅데이터센터 DB 접속 함수"""
    conn = pymysql.connect(
        host = '220.95.212.6',
        user = 'red_dev_jiang',
        password = 'redtable211b',
        db = 'redtable_bigdata_center',
        autocommit = True,
        cursorclass= pymysql.cursors.DictCursor)
    conn.query("set character_set_connection=utf8;")
    conn.query("set character_set_server=utf8;")
    conn.query("set character_set_client=utf8;")
    conn.query("set character_set_results=utf8;")
    conn.query("set character_set_database=utf8;")
    return conn


file_path = os.path.join(os.environ["HOMEPATH"], "Downloads/")

# 식당 한국어 정보

# RT_RESTAURANT_KO_INFO_202407

## https://www.bigdata-culture.kr/bigdata/user/data_market/detail.do?id=19d4465f-02d1-46fa-a651-e6783188fbec

In [2]:
with rt_db_conn() as conn:
    sql = """
    SELECT TABLE_NAME, COLUMN_NAME, column_comment
    FROM information_schema.`COLUMNS`
    WHERE TABLE_NAME = 'RT_RESTAURANT_KO_INFO_202407'
    """
    columns = pd.read_sql(sql, conn)
    
# 컬럼명 가져오기
db_cols = ','.join(columns['COLUMN_NAME'])

In [18]:
# 라이브DB에서는 불필요한 것들을 분류하기 어렵고, 개발DB에서는 지하철 연결이 안되어있어서 나눠서 작업진행함.
with dev_db_conn() as conn:
    sql = """
    SELECT REPLACE(REPLACE(REPLACE(T1.AREA_NM_1, '강원도', '강원특별자치도'), '전라북도', '전북특별자치도'), '제주도', '제주특별자치도') AS 'CTY_NM',
        T3.SRC_ID AS 'RSTRNT_ID', T1.RSTR_NM AS 'RSTRNT_NM',
        IF(T1.RSTR_LNNO_ADRES = '', NULL, T1.RSTR_LNNO_ADRES) AS 'LNM_ADDR',
        IF(T1.RSTR_RDNMADR = '', NULL, T1.RSTR_RDNMADR) AS 'RDNMADR_NM',
       IF(T1.RSTR_TELNO = '', NULL, T1.RSTR_TELNO) AS 'RSTRNT_TEL_NO',
        '' AS 'SUBWAYST_NM', '' AS 'SUBWAYST_DSTNC_VALUE', T1.RSTR_LA AS 'RSTRNT_LA', T1.RSTR_LO AS 'RSTRNT_LO'
    FROM redtable01.RSTR T1
    INNER JOIN redtable01.RSTR_INDEX T2 ON T1.ID = T2.RSTR_ID
    INNER JOIN redtable01.RSTR_CONN T3 ON T2.RSTR_ID = T3.RSTR_ID
    WHERE T2.IS_USE=1 AND T2.IS_OPEN=1 AND T3.SRC='live_db' AND T1.AREA_NM_1 IS NOT NULL AND T1.AREA_NM_1<>''
      AND T1.RSTR_NM IS NOT NULL AND T1.RSTR_NM<>'' AND T1.RSTR_NM NOT LIKE '%%？%%'
    ORDER BY T3.SRC_ID ASC
    """
    df = pd.read_sql(sql, conn)

In [19]:
with live_db_conn() as conn:
    sql = f"""
    SELECT T1.store_id, T2.name, MIN(T1.distance) AS min_distance
    FROM redtable2021.store_landmark T1
    INNER JOIN redtable2021.landmark_master T2 ON T1.landmark_master_id = T2.id
    WHERE T2.`type` = '지하철역' AND T1.store_id IN {tuple(df["RSTRNT_ID"].tolist())}
    GROUP BY T1.store_id
    """
    subway = pd.read_sql(sql, conn)

In [20]:
set_idx = subway.set_index("store_id")["name"].to_dict()
df["SUBWAYST_NM"] = df["RSTRNT_ID"].map(set_idx)

set_idx = subway.set_index("store_id")["min_distance"].to_dict()
df["SUBWAYST_DSTNC_VALUE"] = df["RSTRNT_ID"].map(set_idx)

# 결측치 및 불필요한 데이터 정리
df = df.fillna(np.nan)
df.replace({np.nan:None, '\\n':None}, inplace=True)

In [21]:
df

,CTY_NM,RSTRNT_ID,RSTRNT_NM,LNM_ADDR,RDNMADR_NM,RSTRNT_TEL_NO,SUBWAYST_NM,SUBWAYST_DSTNC_VALUE,RSTRNT_LA,RSTRNT_LO
0,서울특별시,10012,명동정,서울특별시 중구 충무로2가 60-3,서울특별시 중구 삼일대로 299,02-3789-5130,을지로입구역,264.35,37.561597,126.988579
1,서울특별시,10013,명동 서서갈비,서울특별시 중구 명동2가 23,서울특별시 중구 명동10길 35-10,None,을지로입구역,185.92,37.562512,126.985993
2,서울특별시,10014,봉추찜닭,서울특별시 중구 충무로1가 25-8,서울특별시 중구 명동8나길 47,02-3789-9381,시청역,304.28,37.561065,126.982289
3,서울특별시,10016,명동한우방,서울특별시 중구 을지로2가 199-56,서울특별시 중구 명동9길 37-5,02-2267-8132,종각역,528.49,37.56539,126.983946
4,서울특별시,10017,설빙,서울특별시 서대문구 창천동 33-43,서울특별시 서대문구 연세로 23,02-325-8492,신촌역,351.59,37.557762,126.93669
...,...,...,...,...,...,...,...,...,...,...
867213,부산광역시,1191094,회사식당,부산광역시 강서구 지사동 1172,부산광역시 강서구 과학산단로 502,None,None,None,35.150177,128.81681
867214,대구광역시,1191095,후레쉬케터링 모토닉대구공장,대구광역시 달서구 신당동 1095-11,대구광역시 달서구 달서대로 530,053-382-3311,None,None,35.845846,128.493328
867215,부산광역시,1628785,차오란,부산광역시 해운대구 중동 1829,부산광역시 해운대구 달맞이길 30,0507-1356-1253,None,None,35.159937,129.169813
867216,부산광역시,1628786,야키토리 해공,부산광역시 수영구 민락동 24-6,부산광역시 수영구 민락본동로19번길 30-5,0507-1479-8334,None,None,35.158234,129.127114


In [22]:
with rt_db_conn() as conn:
    with conn.cursor() as curs:
        sql = f"""
        INSERT IGNORE INTO redtable_bigdata_center.RT_RESTAURANT_KO_INFO_202407({db_cols})
        VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
        """
        val = df.apply(lambda row:(row["CTY_NM"], row["RSTRNT_ID"], row["RSTRNT_NM"], row["LNM_ADDR"],
                                  row["RDNMADR_NM"], row["RSTRNT_TEL_NO"], row["SUBWAYST_NM"], row["SUBWAYST_DSTNC_VALUE"],
                                  row["RSTRNT_LA"], row["RSTRNT_LO"]), axis=1).tolist()
        curs.executemany(sql, val)